In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim


import sys
ROOT = "../../"
sys.path.append(ROOT) # Add root directory to path

from src.utils.REPClassifier import REPClassifier
from src.utils.StandardDataset import StandardDataset
from src.utils.perturbations import *

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

In [26]:
# Step 1: Load the MNIST dataset
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

# Step 1a: Swap axes to PyTorch's NCHW format
x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

test_history = []

def display_test_history(test_history):
    for i in range(len(test_history)):
        test = test_history[i]
        num_p = len(test['perturbations'])
        io = test['include_original']
        shuffle = test['shuffle']
        baseline = test['baseline_accuracy']
        adversarial = test['adversarial_accuracy']
        adv_str = f"{adversarial[0]*100}/{adversarial[1]*100}/{adversarial[2]*100}/{adversarial[3]*100}"
        print(f"Test {i}: Perturbations ({num_p}), Original ({io}), Shuffle ({shuffle}), Baseline ({baseline*100}%), Adversarial ({adv_str}%)")

In [37]:
noise_multiplier = 2
test_suite = [[["Gaussian", noise_multiplier*0.05]],
              [["Laplacian", noise_multiplier*0.05]],
              [["Lp-norm", noise_multiplier*0.5, 1]],
              [["Lp-norm", noise_multiplier*0.5, 2]],
              [["Lp-norm", noise_multiplier*0.5, 3]],
              [["Lp-norm", noise_multiplier*0.5, 4]],
              [["Lp-norm", noise_multiplier*0.5, float('inf')]],
              [["Gaussian", noise_multiplier*0.05],
               ["Laplacian", noise_multiplier*0.05]],
              [["Gaussian", noise_multiplier*0.05],
               ["Laplacian", noise_multiplier*0.05],
               ["Lp-norm", noise_multiplier*0.5, 2]],
              [["Gaussian", noise_multiplier*0.05],
               ["Gaussian", 2*noise_multiplier*0.05],
               ["Laplacian", noise_multiplier*0.05],
               ["Laplacian", 2*noise_multiplier*0.05],
               ["Lp-norm", noise_multiplier*0.5, 2],
               ["Lp-norm", 2*noise_multiplier*0.5, 2]],
              [["Gaussian", noise_multiplier*0.05],
               ["Laplacian", noise_multiplier*0.05],
               ["Lp-norm", noise_multiplier*0.5, 1],
               ["Lp-norm", noise_multiplier*0.5, 2],
               ["Lp-norm", noise_multiplier*0.5, 3],
               ["Lp-norm", noise_multiplier*0.5, 4],
               ["Lp-norm", noise_multiplier*0.5, float('inf')]],
              [["Gaussian", noise_multiplier*0.05],
               ["Gaussian", 2*noise_multiplier*0.05],
               ["Laplacian", noise_multiplier*0.05],
               ["Laplacian", 2*noise_multiplier*0.05],
               ["Lp-norm", noise_multiplier*0.5, 1],
               ["Lp-norm", 2*noise_multiplier*0.5, 1],
               ["Lp-norm", noise_multiplier*0.5, 2],
               ["Lp-norm", 2*noise_multiplier*0.5, 2],
               ["Lp-norm", noise_multiplier*0.5, 3],
               ["Lp-norm", 2*noise_multiplier*0.5, 3],
               ["Lp-norm", noise_multiplier*0.5, 4],
               ["Lp-norm", 2*noise_multiplier*0.5, 4],
               ["Lp-norm", noise_multiplier*0.5, float('inf')],
               ["Lp-norm", 2*noise_multiplier*0.5, float('inf')]]]

In [54]:
# Step 2: Create the REPClassifier model
channel_widths = [1, 4, 10]  
linear_sizes = [100]
kernel = 5
pooling = nn.AdaptiveMaxPool2d((1, 1))
for noise_multiplier in [5, 10, 15, 20]:
    test_suite = [[["Gaussian", noise_multiplier*0.05]],
                  [["Laplacian", noise_multiplier*0.05]],
                  [["Lp-norm", noise_multiplier*0.5, 1]],
                  [["Lp-norm", noise_multiplier*0.5, 2]],
                  [["Lp-norm", noise_multiplier*0.5, 3]],
                  [["Lp-norm", noise_multiplier*0.5, 4]],
                  [["Lp-norm", noise_multiplier*0.5, float('inf')]],
                  [["Gaussian", noise_multiplier*0.05],
                   ["Laplacian", noise_multiplier*0.05]],
                  [["Gaussian", noise_multiplier*0.05],
                   ["Laplacian", noise_multiplier*0.05],
                   ["Lp-norm", noise_multiplier*0.5, 2]],
                  [["Gaussian", noise_multiplier*0.05],
                   ["Gaussian", 2*noise_multiplier*0.05],
                   ["Laplacian", noise_multiplier*0.05],
                   ["Laplacian", 2*noise_multiplier*0.05],
                   ["Lp-norm", noise_multiplier*0.5, 2],
                   ["Lp-norm", 2*noise_multiplier*0.5, 2]],
                  [["Gaussian", noise_multiplier*0.05],
                   ["Laplacian", noise_multiplier*0.05],
                   ["Lp-norm", noise_multiplier*0.5, 1],
                   ["Lp-norm", noise_multiplier*0.5, 2],
                   ["Lp-norm", noise_multiplier*0.5, 3],
                   ["Lp-norm", noise_multiplier*0.5, 4],
                   ["Lp-norm", noise_multiplier*0.5, float('inf')]],
                  [["Gaussian", noise_multiplier*0.05],
                   ["Gaussian", 2*noise_multiplier*0.05],
                   ["Laplacian", noise_multiplier*0.05],
                   ["Laplacian", 2*noise_multiplier*0.05],
                   ["Lp-norm", noise_multiplier*0.5, 1],
                   ["Lp-norm", 2*noise_multiplier*0.5, 1],
                   ["Lp-norm", noise_multiplier*0.5, 2],
                   ["Lp-norm", 2*noise_multiplier*0.5, 2],
                   ["Lp-norm", noise_multiplier*0.5, 3],
                   ["Lp-norm", 2*noise_multiplier*0.5, 3],
                   ["Lp-norm", noise_multiplier*0.5, 4],
                   ["Lp-norm", 2*noise_multiplier*0.5, 4],
                   ["Lp-norm", noise_multiplier*0.5, float('inf')],
                   ["Lp-norm", 2*noise_multiplier*0.5, float('inf')]]]
    for perturbation_description in test_suite:
        perturbations = create_perturbations(perturbation_description)
        include_original = False
        shuffle = False

        model = REPClassifier(
            channel_widths=channel_widths,
            linear_sizes=linear_sizes,
            kernel=kernel,
            pooling=pooling,
            perturbations=perturbations,
            include_original=include_original,
            shuffle=shuffle,
            num_classes=10,
        )

        # Step 2a: Define the loss function and the optimizer
        lr = 0.01
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Step 3: Create the ART classifier
        classifier = PyTorchClassifier(
            model=model,
            clip_values=(min_pixel_value, max_pixel_value),
            loss=criterion,
            optimizer=optimizer,
            input_shape=(1, 28, 28),
            nb_classes=10,
        )

        # Step 4: Train the ART classifier
        batch_size = 64
        epochs = 10
        classifier.fit(x_train, y_train, batch_size=batch_size, nb_epochs=epochs, verbose=True)
        print(perturbation_description)
        
        # Step 5: Evaluate the ART classifier on benign test examples
        predictions = classifier.predict(x_test)
        baseline_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
        print("Accuracy on benign test examples: {}%".format(baseline_accuracy * 100))

        # Step 6: Generate adversarial test examples
        attack = FastGradientMethod(estimator=classifier, eps=0.1)
        x_test_adv = attack.generate(x=x_test)
        predictions = classifier.predict(x_test_adv)
        adversarial_accuracy1 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
        print("Accuracy on adversarial test examples (epsilon = 0.1): {}%".format(adversarial_accuracy1 * 100))

        attack = FastGradientMethod(estimator=classifier, eps=0.2)
        x_test_adv = attack.generate(x=x_test)
        predictions = classifier.predict(x_test_adv)
        adversarial_accuracy2 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
        print("Accuracy on adversarial test examples (epsilon = 0.2): {}%".format(adversarial_accuracy2 * 100))

        attack = FastGradientMethod(estimator=classifier, eps=0.3)
        x_test_adv = attack.generate(x=x_test)
        predictions = classifier.predict(x_test_adv)
        adversarial_accuracy3 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
        print("Accuracy on adversarial test examples (epsilon = 0.3): {}%".format(adversarial_accuracy3 * 100))

        attack = FastGradientMethod(estimator=classifier, eps=0.5)
        x_test_adv = attack.generate(x=x_test)
        predictions = classifier.predict(x_test_adv)
        adversarial_accuracy4 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
        print("Accuracy on adversarial test examples (epsilon = 0.5): {}%".format(adversarial_accuracy4 * 100))

        test_history.append({'channel_widths': channel_widths,
                            'linear_sizes': linear_sizes,
                            'kernel': kernel,
                            'pooling': pooling,
                            'perturbations': perturbation_description,
                            'include_original': include_original,
                            'shuffle': shuffle,
                            'loss': criterion,
                            'lr': lr,
                            'batch_size': batch_size,
                            'epochs': epochs,
                            'baseline_accuracy': baseline_accuracy,
                            'adversarial_accuracy': [adversarial_accuracy1, adversarial_accuracy2, adversarial_accuracy3, adversarial_accuracy4]})

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Accuracy on benign test examples: 90.05%
Accuracy on adversarial test examples (epsilon = 0.1): 72.3%
Accuracy on adversarial test examples (epsilon = 0.2): 33.09%
Accuracy on adversarial test examples (epsilon = 0.3): 14.030000000000001%
Accuracy on adversarial test examples (epsilon = 0.5): 9.13%


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Accuracy on benign test examples: 85.72999999999999%
Accuracy on adversarial test examples (epsilon = 0.1): 73.13%
Accuracy on adversarial test examples (epsilon = 0.2): 45.48%
Accuracy on adversarial test examples (epsilon = 0.3): 21.34%
Accuracy on adversarial test examples (epsilon = 0.5): 9.33%


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

...

Accuracy on benign test examples: 64.25%
Accuracy on adversarial test examples (epsilon = 0.1): 55.230000000000004%
Accuracy on adversarial test examples (epsilon = 0.2): 39.51%
Accuracy on adversarial test examples (epsilon = 0.3): 25.540000000000003%
Accuracy on adversarial test examples (epsilon = 0.5): 12.76%


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Accuracy on benign test examples: 44.51%
Accuracy on adversarial test examples (epsilon = 0.1): 38.96%
Accuracy on adversarial test examples (epsilon = 0.2): 30.95%
Accuracy on adversarial test examples (epsilon = 0.3): 22.82%
Accuracy on adversarial test examples (epsilon = 0.5): 14.099999999999998%


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [55]:
display_test_history(test_history)

Test 0: Perturbations (0), Original (True), Shuffle (True), Baseline (94.94%), Adversarial (19.6/9.56/8.77/9.11%)
Test 1: Perturbations (1), Original (False), Shuffle (True), Baseline (93.81%), Adversarial (32.67/10.54/7.91/8.23%)
Test 2: Perturbations (1), Original (False), Shuffle (True), Baseline (92.52%), Adversarial (44.440000000000005/11.68/7.9799999999999995/8.08%)
Test 3: Perturbations (1), Original (False), Shuffle (True), Baseline (93.49%), Adversarial (15.73/6.12/5.81/6.4799999999999995%)
Test 4: Perturbations (1), Original (False), Shuffle (True), Baseline (93.08999999999999%), Adversarial (16.98/7.08/5.67/5.1499999999999995%)
Test 5: Perturbations (1), Original (False), Shuffle (True), Baseline (92.80000000000001%), Adversarial (19.46/9.94/7.35/6.34%)
Test 6: Perturbations (1), Original (False), Shuffle (True), Baseline (94.12%), Adversarial (27.38/10.36/9.49/9.5%)
Test 7: Perturbations (1), Original (False), Shuffle (True), Baseline (92.58999999999999%), Adversarial (53.2

In [59]:
best_i = 0
accuracy = 0
eps = 3
for i in range(len(test_history)):
    if test_history[i]['adversarial_accuracy'][eps] > accuracy:
        best_i = i
        accuracy = test_history[i]['adversarial_accuracy'][eps]
print(test_history[best_i])

{'channel_widths': [1, 4, 10], 'linear_sizes': [100], 'kernel': 5, 'pooling': AdaptiveMaxPool2d(output_size=(1, 1)), 'perturbations': [['Gaussian', 1.0]], 'include_original': False, 'shuffle': True, 'loss': CrossEntropyLoss(), 'lr': 0.01, 'batch_size': 64, 'epochs': 10, 'baseline_accuracy': 0.4276, 'adversarial_accuracy': [0.3746, 0.2964, 0.2196, 0.1436]}


In [62]:
import csv

def save_to_csv(data, filename):
    if not data:
        return

    # Extract the keys from the first dictionary as headers
    headers = list(data[0].keys())

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)

        # Write the headers
        writer.writeheader()

        # Write each dictionary as a row in the CSV file
        for item in data:
            writer.writerow(item)

    print(f"Data saved to {filename} successfully.")

save_to_csv(test_history, ROOT + "notebooks/experiment_log/testing_round_1.csv")

Data saved to ../../notebooks/experiment_log/testing_round_1.csv successfully.
